In [2]:
%matplotlib inline
from refer import REFER
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt

# Load Refer Dataset

In [3]:
data_root = './data'  # contains refclef, refcoco, refcoco+, refcocog and images
dataset = 'refclef'
splitBy = 'unc'
refer = REFER(data_root, dataset, splitBy)

loading dataset refclef into memory...
creating index...
index created.
DONE (t=2.95s)


# Stats about the Dataset

In [43]:
# print stats about the given dataset
print (('dataset %s %s contains: ') % (dataset, splitBy))
ref_ids = refer.getRefIds()
image_ids = refer.getImgIds()
print(( '%s expressions for %s refs in %s images.') % (len(refer.Sents), len(ref_ids), len(image_ids)))

print ('\nAmong them:')
if dataset == 'refclef':
    if splitBy == 'unc':
        splits = ['train', 'val', 'testA', 'testB', 'testC']
    else:
        splits = ['train', 'val', 'test']
elif dataset == 'refcoco':
    splits = ['train', 'val', 'test']
elif dataset == 'refcoco+':
    splits = ['train', 'val', 'test']
elif dataset == 'refcocog':
    splits = ['train', 'val']  # we don't have test split for refcocog right now.
    
for split in splits:
    ref_ids = refer.getRefIds(split=split)
    # print('%s refs are in split %s.' % (len(ref_ids), split))
    # print all annotation
    ref_ids = refer.getRefIds(split=split)
    ref_id = ref_ids[2]
    ref = refer.loadAnns(ref_id)
    print('attributes: %s' % ref['attributes'])

dataset refclef unc contains: 
130364 expressions for 99296 refs in 19997 images.

Among them:


KeyError: 2

In [ ]:
# Get annotation IDs for a specific image ID
image_id = 2  # Replace with a valid image ID
ann_ids = refer.getAnnIds(image_ids=[image_id])

# Load the annotations using the obtained annotation IDs
annotations = refer.loadAnns(ann_ids)

# Print the annotations
for ann in annotations:
    print(ann)

# Show Refered Object and its Expressions

In [ ]:
# randomly sample one ref
ref_ids = refer.getRefIds()
ref_id = ref_ids[np.random.randint(0, len(ref_ids))]
ref = refer.Refs[ref_id]
print('ref_id [%s] (ann_id %s)' % (ref_id, refer.refToAnn[ref_id]['id']))
# show the segmentation of the referred object
plt.figure()
# refer.showRef(ref, seg_box='seg')
plt.show()

ref_id [29187] (ann_id 27624_8)


<Figure size 640x480 with 0 Axes>

In [ ]:
# or show the bounding box of the referred object
# refer.showRef(ref, seg_box='box')
plt.show()

In [ ]:
# let's look at the details of each ref
for sent in ref['sentences']:
    print('sent_id[%s]: %s' % (sent['sent_id'], sent['sent']))

sent_id[101470]: sand bottom


In [37]:
ref_id = ref_ids[np.random.randint(0, len(ref_ids))]
ref = refer.Refs[ref_id]
print('ref_id [%s] (ann_id %s)' % (ref_id, refer.refToAnn[ref_id]['id']))
print(ref)

ref_id [49564] (ann_id 48_7)
{'sent_ids': [66159], 'ann_id': '48_7', 'ref_id': 49564, 'image_id': 48, 'split': 'train', 'sentences': [{'tokens': ['blue', 'shirt', 'bottom', 'right'], 'raw': 'blue shirt bottom right', 'sent_id': 66159, 'sent': 'blue shirt bottom right'}], 'category_id': 120}


In [33]:
# random sample one ref

tm = False
while not tm:
    ref_id = ref_ids[np.random.randint(0, len(ref_ids))]
    ref = refer.Refs[ref_id]
    words_length = len(ref['sentences'][0]['tokens'])
    if words_length > 3: # only select the refs with more than 3 words
        print(ref['sentences'][0]['sent'])
        tm = True
        break

white area over between the two guys on the left


sample output of refer dataset:
{'sent_ids': [0], 'ann_id': '19135_1', 'ref_id': 0, 'image_id': 19135, 'split': 'train', 'sentences': [{'tokens': ['sky'], 'raw': 'sky', 'sent_id': 0, 'sent': 'sky'}], 'category_id': 60}

# Convert sentences data into csv

In [ ]:
# {'sent_ids': [0], 'ann_id': '19135_1', 'ref_id': 0, 'image_id': 19135, 'split': 'train', 'sentences': [{'tokens': ['sky'], 'raw': 'sky', 'sent_id': 0, 'sent': 'sky'}], 'category_id': 60} make this json into of sentence id, annotate id, ref id, image id, split, token , raw, sent id, sent, category id


In [44]:
import pandas as pd
import pickle
import os.path as osp

def convert_refer_to_df(data_root, dataset='refcoco', splitBy='unc'):
    """
    Convert REFER data structure to a pandas DataFrame and save as CSV
    
    Args:
        data_root: Path to the data directory
        dataset: Dataset name (default: 'refcoco')
        splitBy: Split by (default: 'unc')
        
    Returns:
        DataFrame containing flattened REFER data
    """
    # Path to the pickle file
    ref_file = osp.join(data_root, dataset, f'refs({splitBy}).p')
    
    # Load the pickle file in binary mode
    refs = pickle.load(open(ref_file, 'rb'))
    
    # Create a list to hold flattened data
    flattened_data = []
    
    # Iterate through each ref
    for ref in refs:
        # For each sentence in the ref
        for sent in ref['sentences']:
            # Create a flattened entry
            entry = {
                'sent_ids_list': ref['sent_ids'],  # This will be a list
                'ann_id': ref['ann_id'],
                'ref_id': ref['ref_id'],
                'image_id': ref['image_id'],
                'split': ref['split'],
                'category_id': ref['category_id'],
                'tokens': sent['tokens'],  # This will be a list
                'raw': sent['raw'],
                'sent_id': sent['sent_id'],
                'sent': sent['sent']
            }
            flattened_data.append(entry)
    
    # Convert to DataFrame
    df = pd.DataFrame(flattened_data)
    
    # Convert list columns to strings for CSV compatibility (optional)
    df['sent_ids_list'] = df['sent_ids_list'].apply(lambda x: ','.join(map(str, x)))
    df['tokens'] = df['tokens'].apply(lambda x: ','.join(map(str, x)))
    
    # Save to CSV
    csv_path = osp.join(data_root, dataset, f'{dataset}_{splitBy}_flattened.csv')
    df.to_csv(csv_path, index=False)
    
    print(f"Data saved to {csv_path}")
    return df

# Example usage
if __name__ == "__main__":
    # Change these paths to match your setup
    data_root = './data'
    dataset = 'refclef'
    splitBy = 'unc'
    
    df = convert_refer_to_df(data_root, dataset, splitBy)
    print(df.head())

Data saved to ./data/refclef/refclef_unc_flattened.csv
  sent_ids_list   ann_id  ref_id  image_id  split  category_id  \
0             0  19135_1       0     19135  train           60   
1             1  19135_2       1     19135  train          235   
2             2  23412_4       2     23412  train          258   
3             3  23412_1       3     23412  train          160   
4             4  23412_2       4     23412  train          120   

                       tokens                         raw  sent_id  \
0                         sky                         sky        0   
1                      statue                      statue        1   
2  anywhere,except,the,people  anywhere except the people        2   
3             person,in,front             person in front        3   
4  person,all,the,way,in,back  person all the way in back        4   

                         sent  
0                         sky  
1                      statue  
2  anywhere except the people  